### Import library

In [1]:
import serial
import threading
from pylsl import StreamInlet, resolve_byprop, local_clock
import keyboard
import csv
import time
import pandas as pd

### Run ldr_recording

In [ ]:
# Serial initialization (Arduino + Photocell)
def init_serial(port='COM5', baudrate=115200):
    try:
        ser = serial.Serial(port, baudrate, timeout=1)
        print(f"Connected to {port} at {baudrate} baud.")
        return ser
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        return None

ser = init_serial()

# Globals
num_block = 1
ldr_data = []              # (relative_time, ldr_value, optional_marker)
markers_applied = []       # (marker_label, relative_time_applied)
raw_marker_data = []       # (timestamp, trigger)
raw_ldr_data = []          # (raw_time, LDR_value)
stop_event = threading.Event()
block_started = False

arduino_start_time = None
lsl_time_at_start = None
start_relative_time = None

latest_ldr_index = -1
ldr_lock = threading.Lock()

# Overflow handling 
last_arduino_time = None
overflow_offset = 0
MAX_MICROS = 2**32  # 32-bit unsigned int wraparound

# Marker Listener Thread (LSL)
def marker_stream_listener():
    global block_started, lsl_time_at_start, latest_ldr_index

    # Waiting 10 seconds for LSL stream
    print("Looking for marker stream...")
    streams = resolve_byprop('type', 'Markers', timeout=10)

    if not streams:
        print("No LSL marker stream found, ending everything.")
        stop_event.set()
        return

    inlet = StreamInlet(streams[0])
    print("Connected to LSL!, Waiting for markers...")

    while not stop_event.is_set():
        sample, timestamp = inlet.pull_sample()
        if sample:
            trigger = sample[0]

            if lsl_time_at_start is None:
                lsl_time_at_start = local_clock()
                print(f"[SYNC] Initialized LSL time reference: {lsl_time_at_start:.6f} s")

            print(f"[Marker] Received: {trigger}")

            with ldr_lock:
                if latest_ldr_index >= 0 and latest_ldr_index < len(ldr_data):
                    old_ts, old_val, _ = ldr_data[latest_ldr_index]
                    ldr_data[latest_ldr_index] = (old_ts, old_val, trigger)
                    markers_applied.append((trigger, old_ts))  # Save marker + time
            raw_marker_data.append((timestamp, trigger))  # Save raw marker data

            if trigger == 'Start Block':
                block_started = True
                print(f"== Start Block {num_block} ==")

            if trigger == 'End Block':
                block_started = False
                print(f"== End Block {num_block} ==")
                stop_event.set()
                break

# Arduino LDR Reader Thread
def read_ldr():
    global block_started, ser
    global arduino_start_time, lsl_time_at_start, start_relative_time, latest_ldr_index
    global last_arduino_time, overflow_offset
    while not stop_event.is_set():
        if block_started and ser and ser.in_waiting:
            try:
                data = ser.readline().strip()
                parts = data.split(b',')
                if len(parts) == 2:
                    raw_time = float(parts[0])
                    ldr_value = parts[1].decode('latin-1')

                    # Handle micros() overflow
                    if last_arduino_time is not None and raw_time < last_arduino_time:
                        overflow_offset += MAX_MICROS
                    last_arduino_time = raw_time
                    arduino_time = raw_time + overflow_offset

                    # First time sync
                    if arduino_start_time is None:
                        arduino_start_time = arduino_time
                        lsl_time_at_start = local_clock()
                        print(f"[SYNC] Arduino time: {arduino_start_time} µs, LSL: {lsl_time_at_start:.6f} s")

                    delta_sec = (arduino_time - arduino_start_time) / 1_000_000.0
                    aligned_time = lsl_time_at_start + delta_sec

                    if start_relative_time is None:
                        start_relative_time = aligned_time

                    relative_time = aligned_time - start_relative_time

                    with ldr_lock:
                        ldr_data.append((relative_time, ldr_value, ''))
                        latest_ldr_index = len(ldr_data) - 1

                    raw_ldr_data.append((raw_time, ldr_value))  # Save raw LDR data

                else:
                    print(f"[WARN] Invalid format: {data}")

            except Exception as e:
                print(f"[ERROR] {e} from data: {data}")

# ESC key interrupt
def check_for_esc():
    while not stop_event.is_set():
        if keyboard.is_pressed('esc'):
            print("ESC pressed, stopping everything.")
            stop_event.set()
        time.sleep(0.1)

# Save LDR to CSV
def save_data_to_csv(filename):
    filename = filename + '_process.csv'
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time (s from start)', 'LDR Value', 'Marker'])

        for ts, val, label in ldr_data:
            writer.writerow([f"{ts:.6f}", val, label])

    print(f"LDR data saved to {filename}")

# Save Markers to CSV 
def save_markers_to_csv(filename):
    filename = filename + '_marker.csv'
    with open(filename , mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Marker Label', 'Time (s from start)'])

        for label, ts in markers_applied:
            writer.writerow([label, f"{ts:.6f}"])

    print(f"Marker timestamps saved to {filename}")

# Save Raw Data to CSV
def save_raw_data_to_csv(filename):
    filename = filename + '_raw.csv'
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Timestamp', 'Value', 'Type'])

        for ts, val in raw_ldr_data:
            writer.writerow([f"{ts:.6f}", val, 'LDR'])
        for ts, trigger in raw_marker_data:
            writer.writerow([f"{ts:.6f}", trigger, 'Marker'])

    print(f"Raw data saved to {filename}")

# Run Threads
listener_thread = threading.Thread(target=marker_stream_listener)
ldr_thread = threading.Thread(target=read_ldr)
esc_thread = threading.Thread(target=check_for_esc, daemon=True)

listener_thread.start()
ldr_thread.start()
esc_thread.start()

listener_thread.join()
ldr_thread.join()

print(f"\n[SUMMARY] Collected {len(ldr_data)} LDR samples and {len(markers_applied)} markers applied.")

file_name = 'test101'
save_data_to_csv(file_name)
save_markers_to_csv(file_name)
save_raw_data_to_csv(file_name)

if ser:
    ser.close()


Connected to COM5 at 115200 baud.
Looking for marker stream...
Connected to LSL! Waiting for markers...
[SYNC] Initialized LSL time reference: 153269.262993 s
[Marker] Received: Start Block
== Start Block 1 ==
[ERROR] could not convert string to float: b'925\xf82216' from data: b'925\xf82216,301'
[SYNC] Arduino time: 4120.0 µs, LSL: 153269.265478 s
[Marker] Received: trial_1
[Marker] Received: end_1
[Marker] Received: trial_2
[Marker] Received: end_2
[Marker] Received: trial_3
[Marker] Received: end_3
[Marker] Received: trial_4
[Marker] Received: end_4
[Marker] Received: trial_5
[Marker] Received: end_5
[Marker] Received: trial_6
[Marker] Received: end_6
[Marker] Received: trial_7
[Marker] Received: end_7
[Marker] Received: trial_8
[Marker] Received: end_8
[Marker] Received: trial_9
[Marker] Received: end_9
[Marker] Received: trial_10
[Marker] Received: end_10
[Marker] Received: trial_11
[Marker] Received: end_11
[Marker] Received: trial_12
[Marker] Received: end_12
[Marker] Received: 